In [1]:
# Import the necessary modules and libraries
import pandas as pd
import numpy as np
import pickle
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten
from sklearn.preprocessing import MinMaxScaler

In [2]:
#define some helper functions
def create_model(N):
    model = keras.Sequential()
    model.add(keras.layers.Dense(units = 3, activation = 'linear', input_shape=[3]))
    for i in range(N):
        model.add(keras.layers.Dense(units = 64, activation = 'relu')) #elu might also be good
    model.add(keras.layers.Dense(units = 1, activation = 'linear'))
    model.compile(loss='mse', optimizer="adam")
    return model

def scale_data(unscaled_x):
    with open('scaler.pkl', 'rb') as handle:
        scaler = pickle.load(handle)
    return scaler.transform(unscaled_x)

def fit_ensemble(vispath,unscaled_x):
    tf.get_logger().setLevel('ERROR')
    models = glob.glob(vispath+'/*')                         
    scaled_x = scale_data(unscaled_x)
    allpreds = []
    for i in range(len(models)):
        new_model = create_model(6)
        new_model.load_weights(models[i]) #get corresponding file
        pred_log_albedos = new_model.predict(scaled_x).flatten()#return prediction for sorted input
        allpreds.append(pred_log_albedos)
    allpreds = np.array(allpreds)
    return np.mean(allpreds,axis=0), np.std(allpreds,axis=0)

def fit_belt(model,unscaled_x):
    scaled_x = scale_data(unscaled_x)
    new_model = create_model(6)
    new_model.load_weights(model) #get corresponding file
    pred = new_model.predict(scaled_x).flatten()#return prediction for sorted input
    return pred * np.sqrt(np.pi/2)


### Let's compare the neural net to the catalog

The proper elements of 3 Juno are, a = 2.66929, e = 0.2320, sin(I) = 0.2308

The proper elements of 5 Astraea are, a = 2.57621, e = 0.2278, sin(I) = 0.0788

In [3]:
elements = np.array([[2.66929, 0.2320, 0.2308],[2.57621,0.2278,0.0788]])

### We can fit the just the visible or infrared ensenbles individually by providing the elements and the location of the ensemble weights

In [4]:
fit_ensemble('./vis_ensemble',elements)

(array([-0.7735125, -0.8591196], dtype=float32),
 array([0.05978905, 0.04262035], dtype=float32))

### We can also find the standard deviation of the belt wrt either albedo, by pointing to the correct weight file

In [5]:
fit_belt('./std_belt_models/bestmodel_vis.h5',elements)

array([0.31164357, 0.4230555 ], dtype=float32)

### finally we can compare this to the cataloge and see they are identical to the values for asteroids number 3 & 5

In [8]:
pd.read_csv('final_catalogue.csv').iloc[1:4,:]

,no,sigma_pred_vis,pred_vis,sigma_belt_vis,sigma_pred_ir,pred_ir,sigma_belt_ir
1,3,0.059789,-0.773513,0.311644,0.060646,-0.598993,0.321617
2,4,0.029367,-0.495357,0.154884,0.028580,-0.323304,0.120736
3,5,0.042620,-0.859120,0.423056,0.041148,-0.682643,0.401361
